# IMDb Top 250

### Full project Task :  
1. Scrape data from a IMDb website ( IMDb Top 250) using BeautifulSoup.  *(used **SELENIUM** instead because Beautiful a js-dynamically generated list)* 
2. Clean it with **Pandas**
3. load it into **MS SQL** for analysis; and
4. visualize trends in **Power BI**.


### Tools
  - **Jupyter Notenbook***
      - libraries:
          - Pandas,
          - Selenium,
          - time
            
  - **Microsoft Excel**
  - **Microsoft Sql server** 
  - **Microsoft Power BI**


In [133]:
#import libraries
import pandas as pd
from selenium import webdriver # the library itself
from selenium.webdriver.common.by import By  #key to selecting html/css tags
from selenium.webdriver.chrome.options import Options #used  for debugging, and setting up  headless chrome
import time #for asynchronous operation to load up the entire list info 


#data cleaning
import numpy as np
import re


# WEB SCRAPING

In [9]:
# IMDb Top 250 URL
url ="https://www.imdb.com/chart/top/"


#setting up chrome

options = Options()
options.add_argument('--disable-gpu')
options.add_argument('--no-sandbox')

driver =  webdriver.Chrome(options=options)
driver.get(url) # to Open the IMDb Top 250 page




#empty list to append scrapped data
movies_dataset = []



try:
    time.sleep(3)

    #grab and save all 25 movie links first
    movies_details_links = driver.find_elements(
        By.CSS_SELECTOR, "ul.ipc-metadata-list li.ipc-metadata-list-summary-item a.ipc-title-link-wrapper"
    )

    print(f"\nTotal links found: {len(movies_details_links)}\n")
    
    movie_links = []
    for el in movies_details_links:
        href = el.get_attribute('href')
        if href and "/title/" in href:
            # Extract clean movie ID URL (remove tracking parameters)
            base_url = href.split('?')[0]
            movie_links.append(base_url)

    print(f"Number of movie links: {len(movie_links)}\n")

    # Loop through all 250 movies
    for rank_no, link in enumerate(movie_links, 1): 
        #print(f"\n🔗 Processing movie #{rank_no}: {link}")
        driver.get(link)
        time.sleep(2)

        # Title
        try:
            title = driver.find_element(By.CSS_SELECTOR, 'h1').text
        except:
            title = 'N/A'

        # Year Released
        try:
            year_released = driver.find_element(
                By.CSS_SELECTOR, 
                'ul[data-testid="hero-title-block__metadata"] li:first-child a'
            ).text
        except:
            try:
                year_released = driver.find_element(
                    By.XPATH, 
                    '//a[contains(@href, "releaseinfo")]'
                ).text
            except:
                year_released = 'N/A'

        # Runtime
        try:
            runtime = driver.find_element(
                By.CSS_SELECTOR, 
                'ul[data-testid="hero-title-block__metadata"] li:nth-child(3)'
            ).text
        except:
            try:
                # Fallback for different layout
                runtime = driver.find_element(
                    By.CSS_SELECTOR, 
                    'li[data-testid="title-techspec_runtime"] .ipc-metadata-list-item__content-container'
                ).text
            except:
                runtime = 'N/A'

        # MPAA Rating
        try:
            mpaa_rating = [rating.text for rating in driver.find_elements(By.CSS_SELECTOR, "li.ipc-inline-list__item")]
            mpaa_rating = mpaa_rating[5] 
        except:
            mpaa_rating = 'N/A'

        # IMDb Rating
        try:
            imdb_rating = driver.find_element(
                By.CSS_SELECTOR, 
                'div[data-testid="hero-rating-bar__aggregate-rating__score"] span:first-child'
            ).text
        except:
            imdb_rating = 'N/A'

        # Director(s)
        try:
            # Find the director section by label
            director_label = driver.find_element(
                By.XPATH, 
                '//div[@class="ipc-metadata-list-item__label" and contains(text(), "Director")]'
            )
    
            # Find the container with director names
            director_container = director_label.find_element(
                By.XPATH, 
                './following-sibling::div[contains(@class, "ipc-metadata-list-item__content-container")]'
            )
    
            # Extract all director names
            directors = director_container.find_elements(
                By.CSS_SELECTOR, 
                'a[href*="/name/"]'
            )
    
            director = ", ".join([d.text for d in directors if d.text])
        except:
            try:
                # Fallback method using data-testid
                directors = driver.find_elements(
                    By.CSS_SELECTOR, 
                    'li[data-testid="title-pc-principal-credit"]:first-child a'
                )
                director = ", ".join([d.text for d in directors if d.text])
            except:
                director = 'N/A'
            
        # Movie Poster URL
        try:
            # First try to get the poster from the hero section
            poster = driver.find_element(
                By.CSS_SELECTOR, 
                'div[data-testid="hero-media__poster"] img'
            )
            poster_url = poster.get_attribute('src')
        except:
            try:
                # Fallback to the poster container
                poster = driver.find_element(
                    By.CSS_SELECTOR, 
                    'div.ipc-poster img'
                )
                poster_url = poster.get_attribute('src')
            except:
                try:
                    # Try to get from media image
                    poster = driver.find_element(
                        By.CSS_SELECTOR, 
                        'div.ipc-media img'
                    )
                    poster_url = poster.get_attribute('src')
                except:
                    poster_url = 'N/A'

        # Stars (top 5)
        try:
            stars = driver.find_elements(
                By.CSS_SELECTOR, 
                'div[data-testid="title-cast-item"] a[data-testid="title-cast-item__actor"]'
            )
            star_names = ", ".join([s.text for s in stars[:5]])
        except:
            star_names = 'N/A'

        # Country
        try:
            country = driver.find_element(
                By.XPATH, 
                '//li[@data-testid="title-details-origin"]//div/ul/li/a'
            ).text
        except:
            country = 'N/A'

        # Language
        try:
            languages = driver.find_elements(
                By.XPATH, 
                '//li[@data-testid="title-details-languages"]//li/a'
            )
            language = ", ".join([lang.text for lang in languages])
        except:
            language = 'N/A'

        # Budget
        try:
            budget = driver.find_element(
                By.XPATH, 
                '//li[@data-testid="title-boxoffice-budget"]//div/ul/li/span'
            ).text
        except:
            budget = 'N/A'

        # Gross Worldwide
        try:
            gross = driver.find_element(
                By.XPATH, 
                '//li[@data-testid="title-boxoffice-cumulativeworldwidegross"]//div/ul/li/span'
            ).text
        except:
            gross = 'N/A'

        #append result into the 'movies_dataset'
        movies_dataset.append({
                'IMDb Ranking': rank_no,
                'Title': title,
                'Year Release': year_released,
                'Age Rating': mpaa_rating,
                'Runtime': runtime,
                'IMDb Rating': imdb_rating,
                'Country': country,
                'Director(s)': director,
                'Top Casts': star_names,
                'Original Language': language,
                'Budget' : budget,
                'Gross (Worldwide)' : gross,
                'Poster': poster_url,
                'Movie URL': link
            })


except Exception as e:
    print("Error:", e)
    import traceback
    traceback.print_exc()

finally:
    driver.quit()
    print("✅ Done.")



Total links found: 250

Number of movie links: 250

✅ Done.


In [4]:
movies_dataset

In [21]:
df = pd.DataFrame(movies_dataset)

df.head()

,IMDb Ranking,Title,Year Release,Age Rating,Runtime,IMDb Rating,Country,Director(s),Top Casts,Original Language,Budget,Gross (Worldwide),Poster,Movie URL
0,1,The Shawshank Redemption,1994,R,2 hours 22 minutes,9.3,United States,Frank Darabont,"Tim Robbins, Morgan Freeman, Bob Gunton, Willi...",English,"$25,000,000 (estimated)","$29,333,735",https://m.media-amazon.com/images/M/MV5BMDAyY2...,https://www.imdb.com/title/tt0111161/
1,2,The Godfather,1972,R,2 hours 55 minutes,9.2,United States,Francis Ford Coppola,"Marlon Brando, Al Pacino, James Caan, Diane Ke...","English, Italian, Latin","$6,000,000 (estimated)","$250,342,198",https://m.media-amazon.com/images/M/MV5BNGEwYj...,https://www.imdb.com/title/tt0068646/
2,3,The Dark Knight,2008,PG-13,2 hours 32 minutes,9.0,United States,Christopher Nolan,"Christian Bale, Heath Ledger, Aaron Eckhart, M...","English, Mandarin","$185,000,000 (estimated)","$1,012,558,979",https://m.media-amazon.com/images/M/MV5BMTMxNT...,https://www.imdb.com/title/tt0468569/
3,4,The Godfather Part II,1974,R,3 hours 22 minutes,9.0,United States,Francis Ford Coppola,"Al Pacino, Robert De Niro, Robert Duvall, Dian...","English, Italian, Spanish, Latin, Sicilian","$13,000,000 (estimated)","$47,983,687",https://m.media-amazon.com/images/M/MV5BMDIxMz...,https://www.imdb.com/title/tt0071562/
4,5,12 Angry Men,1957,Approved,1 hour 36 minutes,9.0,United States,Sidney Lumet,"Henry Fonda, Lee J. Cobb, Martin Balsam, John ...",English,"$350,000 (estimated)","$2,945",https://m.media-amazon.com/images/M/MV5BYjE4Nz...,https://www.imdb.com/title/tt0050083/


In [25]:
df.to_csv('imdb_top_250.csv', index=False)

# DATA CLEANING

In [365]:
#import the csv 

df_imdb = pd.read_csv('imdb_top_250.csv')

In [367]:
df_imdb.head(3)

,IMDb Ranking,Title,Year Release,Age Rating,Runtime,IMDb Rating,Country,Director(s),Top Casts,Original Language,Budget,Gross (Worldwide),Poster,Movie URL
0,1,The Shawshank Redemption,1994,R,2 hours 22 minutes,9.3,United States,Frank Darabont,"Tim Robbins, Morgan Freeman, Bob Gunton, Willi...",English,"$25,000,000 (estimated)","$29,333,735",https://m.media-amazon.com/images/M/MV5BMDAyY2...,https://www.imdb.com/title/tt0111161/
1,2,The Godfather,1972,R,2 hours 55 minutes,9.2,United States,Francis Ford Coppola,"Marlon Brando, Al Pacino, James Caan, Diane Ke...","English, Italian, Latin","$6,000,000 (estimated)","$250,342,198",https://m.media-amazon.com/images/M/MV5BNGEwYj...,https://www.imdb.com/title/tt0068646/
2,3,The Dark Knight,2008,PG-13,2 hours 32 minutes,9.0,United States,Christopher Nolan,"Christian Bale, Heath Ledger, Aaron Eckhart, M...","English, Mandarin","$185,000,000 (estimated)","$1,012,558,979",https://m.media-amazon.com/images/M/MV5BMTMxNT...,https://www.imdb.com/title/tt0468569/


In [369]:
#columns
df_imdb.columns

Index(['IMDb Ranking', 'Title', 'Year Release', 'Age Rating', 'Runtime',
       'IMDb Rating', 'Country', 'Director(s)', 'Top Casts',
       'Original Language', 'Budget', 'Gross (Worldwide)', 'Poster',
       'Movie URL'],
      dtype='object')

In [371]:
#check the data types of each column
df_imdb.dtypes

IMDb Ranking           int64
Title                 object
Year Release           int64
Age Rating            object
Runtime               object
IMDb Rating          float64
Country               object
Director(s)           object
Top Casts             object
Original Language     object
Budget                object
Gross (Worldwide)     object
Poster                object
Movie URL             object
dtype: object

In [373]:
##------------------------------------------------------------------------

#### a. fix 'runtime column'

In [376]:
# Convert the duration/runtime of the movie from 'x hours yy minutes' to  'zz minutes

def to_minutes(runtime):
    runtime = runtime.lower()
    minutes = 0

    if "hour" in runtime:
        hour_part = runtime.split("hour")[0].strip()
        hours = int(hour_part.split()[-1])
        minutes += hours * 60

    if "minute" in runtime:
        minute_part = runtime.split("minute")[0].strip()
        mins = int(minute_part.split()[-1])
        # Avoid double-counting if hour is also present
        if "hour" in runtime:
            mins = int(minute_part.split()[-1])
        minutes += mins

    return minutes

df_imdb['Runtime_minutes'] = df_imdb['Runtime'].apply(to_minutes)


In [378]:

df_imdb[['Runtime_minutes', 'Title']].head()

,Runtime_minutes,Title
0,142,The Shawshank Redemption
1,175,The Godfather
2,152,The Dark Knight
3,202,The Godfather Part II
4,96,12 Angry Men


In [380]:
#drop previous duration (runtime)column 
df_imdb.drop(['Runtime'], axis=1, inplace=True)

In [382]:
df_imdb.head(2)

,IMDb Ranking,Title,Year Release,Age Rating,IMDb Rating,Country,Director(s),Top Casts,Original Language,Budget,Gross (Worldwide),Poster,Movie URL,Runtime_minutes
0,1,The Shawshank Redemption,1994,R,9.3,United States,Frank Darabont,"Tim Robbins, Morgan Freeman, Bob Gunton, Willi...",English,"$25,000,000 (estimated)","$29,333,735",https://m.media-amazon.com/images/M/MV5BMDAyY2...,https://www.imdb.com/title/tt0111161/,142
1,2,The Godfather,1972,R,9.2,United States,Francis Ford Coppola,"Marlon Brando, Al Pacino, James Caan, Diane Ke...","English, Italian, Latin","$6,000,000 (estimated)","$250,342,198",https://m.media-amazon.com/images/M/MV5BNGEwYj...,https://www.imdb.com/title/tt0068646/,175


In [384]:
#---------------------------------------------------------------------------------------------

#### b. fix 'Budget column'

In [387]:
# 1. cleaning the budget column  
# 2.converting all currencies to USD 

In [389]:
# To show all rows
#pd.set_option('display.max_rows', None)

df_imdb["Budget"].head()

0     $25,000,000 (estimated)
1      $6,000,000 (estimated)
2    $185,000,000 (estimated)
3     $13,000,000 (estimated)
4        $350,000 (estimated)
Name: Budget, dtype: object

In [391]:

# Comprehensive exchange rates (approximate values - update as needed)
EXCHANGE_RATES = {
    # Major currencies
    '$': 1.00,       # US Dollar
    'A$': 0.67,      # Australian Dollar
    'R$': 0.19,      # Brazilian Real
    'C$': 0.75,      # Canadian Dollar
    '€': 1.12,       # Euro
    '£': 1.27,       # British Pound
    '¥': 0.0068,     # Japanese Yen
    '₹': 0.012,      # Indian Rupee
    '₩': 0.00085,    # South Korean Won
    
    # Country codes
    'ITL': 0.0006,   # Italian Lira
    'DEM': 0.56,     # German Mark
    'DKK': 0.14,     # Danish Krone
    
    # Fallbacks
    'US': 1.00,      # US Dollar (alternative)
    'AU': 0.67,      # Australian Dollar
    'CA': 0.75,      # Canadian Dollar
    'EU': 1.12,      # Euro
    'UK': 1.27,      # British Pound
    'JP': 0.0068,    # Japanese Yen
    'IN': 0.012,     # Indian Rupee
    'KR': 0.00085,   # South Korean Won
}

def convert_budget_to_usd(budget_str):
    if pd.isna(budget_str):
        return np.nan
        
    budget_str = str(budget_str).strip()
    
    # Step 1: Extract numeric value
    amount_match = re.search(r'[\d,]+', budget_str)
    if not amount_match:
        return np.nan
        
    amount_str = amount_match.group(0).replace(',', '')
    try:
        amount = float(amount_str)
    except:
        return np.nan
        
    # Step 2: Identify currency
    currency = '$'  # Default to USD
    
    # Look for currency symbols
    if '₹' in budget_str:
        currency = '₹'
    elif '₩' in budget_str:
        currency = '₩'
    elif '€' in budget_str:
        currency = '€'
    elif '£' in budget_str:
        currency = '£'
    elif '¥' in budget_str:
        currency = '¥'
    # Look for currency codes
    elif 'ITL' in budget_str:
        currency = 'ITL'
    elif 'DEM' in budget_str:
        currency = 'DEM'
    elif 'DKK' in budget_str:
        currency = 'DKK'
    # Look for country prefixes
    elif budget_str.startswith('A$'):
        currency = 'A$'
    elif budget_str.startswith('R$'):
        currency = 'R$'
    elif budget_str.startswith('C$'):
        currency = 'C$'
        
    # Step 3: Convert to USD
    rate = EXCHANGE_RATES.get(currency, 1.0)  # Default to USD if unknown
    return amount * rate

# Apply conversion
df_imdb['Budget_USD'] = df_imdb['Budget'].apply(convert_budget_to_usd)


In [393]:
#drop  Budget column
df_imdb.drop(['Budget'], axis=1, inplace=True)

In [395]:
df_imdb['Budget_USD']

0      2.500000e+07
1      6.000000e+06
2      1.850000e+08
3      1.300000e+07
4      3.500000e+05
5      9.400000e+07
6      2.200000e+07
7      8.000000e+06
8      9.300000e+07
9      1.200000e+06
10     5.500000e+07
11     9.400000e+07
12     6.300000e+07
13     1.600000e+08
14     1.800000e+07
15     6.300000e+07
16     2.500000e+07
17     1.650000e+08
18     3.000000e+06
19     3.300000e+07
20     3.180000e+06
21     1.900000e+07
22     8.500000e+05
23     7.000000e+07
24     6.270000e+05
25     6.000000e+07
26     9.000000e+06
27     1.020000e+08
28     1.100000e+07
29     1.900000e+07
30     1.900000e+07
31     3.500000e+07
32     1.030000e+08
33     1.140000e+07
34     8.069470e+05
35     4.500000e+07
36     3.700000e+06
37     9.000000e+07
38     3.300000e+06
39              NaN
40     4.000000e+07
41     2.000000e+07
42     1.600000e+07
43     1.000000e+08
44     9.500000e+05
45     6.000000e+06
46     1.064000e+07
47     5.000000e+06
48     1.100000e+07
49     1.500000e+06


In [397]:
df_imdb.head(3)

,IMDb Ranking,Title,Year Release,Age Rating,IMDb Rating,Country,Director(s),Top Casts,Original Language,Gross (Worldwide),Poster,Movie URL,Runtime_minutes,Budget_USD
0,1,The Shawshank Redemption,1994,R,9.3,United States,Frank Darabont,"Tim Robbins, Morgan Freeman, Bob Gunton, Willi...",English,"$29,333,735",https://m.media-amazon.com/images/M/MV5BMDAyY2...,https://www.imdb.com/title/tt0111161/,142,25000000.0
1,2,The Godfather,1972,R,9.2,United States,Francis Ford Coppola,"Marlon Brando, Al Pacino, James Caan, Diane Ke...","English, Italian, Latin","$250,342,198",https://m.media-amazon.com/images/M/MV5BNGEwYj...,https://www.imdb.com/title/tt0068646/,175,6000000.0
2,3,The Dark Knight,2008,PG-13,9.0,United States,Christopher Nolan,"Christian Bale, Heath Ledger, Aaron Eckhart, M...","English, Mandarin","$1,012,558,979",https://m.media-amazon.com/images/M/MV5BMTMxNT...,https://www.imdb.com/title/tt0468569/,152,185000000.0


In [399]:
#=-----------------------------------------------------

#### a. fix 'Gross (Worldwide) column'

In [402]:
df_imdb["Gross (Worldwide)"]

0         $29,333,735
1        $250,342,198
2      $1,012,558,979
3         $47,983,687
4              $2,945
5      $1,138,585,547
6        $322,161,245
7        $213,928,762
8        $888,483,037
9         $25,266,106
10       $678,226,465
11       $938,532,865
12       $101,321,009
13       $839,381,898
14       $550,016,086
15       $467,841,735
16        $47,068,004
17       $758,614,115
18       $109,115,366
19       $328,981,827
20        $10,635,461
21       $272,742,922
22         $1,082,148
23       $482,352,390
24        $30,681,033
25       $286,801,374
26       $230,099,013
27       $517,778,573
28       $775,398,507
29       $385,053,307
30       $358,926,964
31       $120,098,945
32       $465,516,248
33       $262,616,458
34        $32,251,723
35       $979,161,373
36           $841,612
37       $291,481,358
38        $50,360,880
39            $15,222
40       $109,676,311
41        $23,875,714
42        $20,330,788
43       $690,824,738
44         $4,729,846
45        

In [404]:
# Remove dollar signs and commas, then convert to float

df_imdb["Gross (Worldwide)"] = pd.to_numeric(
    df_imdb["Gross (Worldwide)"].replace('[\$,]', '', regex=True),
    errors='coerce'
)

<>:4: SyntaxWarning: invalid escape sequence '\$'
<>:4: SyntaxWarning: invalid escape sequence '\$'
C:\Users\USER\AppData\Local\Temp\ipykernel_4104\3068640748.py:4: SyntaxWarning: invalid escape sequence '\$'
  df_imdb["Gross (Worldwide)"].replace('[\$,]', '', regex=True),


In [406]:
df_imdb["Gross_USD"] =df_imdb["Gross (Worldwide)"]

In [408]:
#delete 'Gross (Worldwide)' column

df_imdb.drop(columns=['Gross (Worldwide)'], inplace=True)

In [410]:
df_imdb.head()

,IMDb Ranking,Title,Year Release,Age Rating,IMDb Rating,Country,Director(s),Top Casts,Original Language,Poster,Movie URL,Runtime_minutes,Budget_USD,Gross_USD
0,1,The Shawshank Redemption,1994,R,9.3,United States,Frank Darabont,"Tim Robbins, Morgan Freeman, Bob Gunton, Willi...",English,https://m.media-amazon.com/images/M/MV5BMDAyY2...,https://www.imdb.com/title/tt0111161/,142,25000000.0,2.933374e+07
1,2,The Godfather,1972,R,9.2,United States,Francis Ford Coppola,"Marlon Brando, Al Pacino, James Caan, Diane Ke...","English, Italian, Latin",https://m.media-amazon.com/images/M/MV5BNGEwYj...,https://www.imdb.com/title/tt0068646/,175,6000000.0,2.503422e+08
2,3,The Dark Knight,2008,PG-13,9.0,United States,Christopher Nolan,"Christian Bale, Heath Ledger, Aaron Eckhart, M...","English, Mandarin",https://m.media-amazon.com/images/M/MV5BMTMxNT...,https://www.imdb.com/title/tt0468569/,152,185000000.0,1.012559e+09
3,4,The Godfather Part II,1974,R,9.0,United States,Francis Ford Coppola,"Al Pacino, Robert De Niro, Robert Duvall, Dian...","English, Italian, Spanish, Latin, Sicilian",https://m.media-amazon.com/images/M/MV5BMDIxMz...,https://www.imdb.com/title/tt0071562/,202,13000000.0,4.798369e+07
4,5,12 Angry Men,1957,Approved,9.0,United States,Sidney Lumet,"Henry Fonda, Lee J. Cobb, Martin Balsam, John ...",English,https://m.media-amazon.com/images/M/MV5BYjE4Nz...,https://www.imdb.com/title/tt0050083/,96,350000.0,2.945000e+03


In [412]:
df_imdb[['Budget_USD','Gross_USD']].describe()

,Budget_USD,Gross_USD
count,2.300000e+02,2.450000e+02
mean,3.765647e+07,2.305979e+08
std,5.680383e+07,3.709631e+08
min,1.330000e+05,3.990000e+02
25%,3.000000e+06,7.971440e+06
50%,1.500000e+07,6.441700e+07
75%,4.750000e+07,3.217527e+08
max,3.560000e+08,2.799439e+09


In [414]:
df_imdb.isna().sum()

IMDb Ranking          0
Title                 0
Year Release          0
Age Rating            0
IMDb Rating           0
Country               0
Director(s)           0
Top Casts             0
Original Language     1
Poster                0
Movie URL             0
Runtime_minutes       0
Budget_USD           20
Gross_USD             5
dtype: int64

# Save Cleaned Data 


In [421]:
df_imdb.to_csv('imdb_top_250_cleaned.csv',  index=False)